In [1]:
from insolAPI.WebAPI import API
import simplejson as json
import pandas as pd
import pendulum as pdl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
from tqdm import tqdm
import sys
from tabulate import tabulate
from pytz import timezone

In [2]:
time_args = dict(
    start=pdl.yesterday().subtract(weeks=1).to_datetime_string(),
    stop=(pdl.now()).to_datetime_string(),
    timezone = timezone('Europe/Zurich')
)

def read_json_config():
    """
    Read the config.json file and return the api key
    """
    with open("config.json") as f:
        config_data = json.load(f)

    installations = {}

    # Iterate through locations
    locations = config_data['locations']
    for location in locations:
        # Store details in a dictionary
        location_details = {
            'id': location['id'],
            'name': location['name'],
            'latitude': location['latitude'],
            'longitude': location['longitude'],
            'wind_threshold': location['wind_threshold'],
            'high_temperature_threshold': location['high_temperature_threshold'],
            'has_a_screen': location['has_a_screen'],
        }
        installations[location['name']] = location_details
    return installations, [config_data['api_key'], config_data['api_url']]


installation_path = "C:/Users/Insolight/Desktop/InsolReports/Scripts"
with open(installation_path + "/local.json") as f:
    local_data = json.load(f)
api = API(local_data["API_user"], local_data["API_pwd"], dev_space=False)

list_sensor = [
    "PAR",
    "IRRAD",
    "GII",
    "DNI",
    "DHI",
    "TEMP",
    "HUMI",
    "RAIN",
    "RAIN_TYPE",
    "RAIN_ACCUMULATED",
    "WIND",
    "WIND_DIR",
    "VIRTUAL",
    "LEAF_TEMP"
]

dict_instal = {
    "Etchelecu": 26,
    "Agroscope Series": 24,
    "Agroscope Berries": 25,
    "Bioschmid": 23,
}

In [4]:
dict_instal[instal]

NameError: name 'instal' is not defined

In [24]:
logs_joined = {}
for instal in tqdm(dict_instal):
    logs_joined[instal] = {}
    sensor_number = 0
    for sensor_type in api.SensorsTypes:
        if str(sensor_type).split(".")[1] in list_sensor:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                logs_joined[instal][list_sensor[sensor_number]] = api.get_sensor_channels_logs_joined(**time_args,sensor_type=sensor_type, install=dict_instal[instal])
                sensor_number += 1

100%|██████████| 4/4 [00:38<00:00,  9.53s/it]


In [23]:
unique_sensors = {}
logs_joined_unique = {}
last_log = {}
for instal in dict_instal:
    unique_sensors[instal] = []
    for sensor_type in logs_joined[instal]:
        try:
            unique_sensors[instal].extend(logs_joined[instal][sensor_type]["sensor_name"].unique())
        except :
            pass

    logs_joined_unique[instal] = {}
    for sensor_type in logs_joined[instal]:
        for unique_sensor in unique_sensors[instal]:
            try:
                if unique_sensor in logs_joined[instal][sensor_type]["sensor_name"].unique():
                    logs_joined_unique[instal][unique_sensor] = logs_joined[instal][sensor_type].loc[logs_joined[instal][sensor_type]["sensor_name"] == unique_sensor]
                    #round the index to the nearest minute
                    logs_joined_unique[instal][unique_sensor].index = logs_joined_unique[instal][unique_sensor].index.round('min')
                    logs_joined_unique[instal][unique_sensor] = logs_joined_unique[instal][unique_sensor].index.drop_duplicates(keep='last')
                    # remove duplicates based on index and sensor_name
                    # logs_joined_unique[instal][unique_sensor] = logs_joined_unique[instal][unique_sensor][~logs_joined_unique[instal][unique_sensor].duplicated(subset=['sensor_name', logs_joined_unique[instal][unique_sensor].index], keep='last')]
                    # print(unique_sensor)
            except :
                pass

    for sensor in logs_joined_unique[instal]:
        logs_joined_unique[instal][sensor] = logs_joined_unique[instal][sensor].dropna(subset=[logs_joined_unique[instal][sensor].columns[1]])

    last_log[instal] = {}
    for sensor in logs_joined_unique[instal]:
        last_log[instal][sensor] = logs_joined_unique[instal][sensor].index[-1]
        if last_log[instal][sensor] > pdl.now().subtract(minutes=10):
            last_log[instal][sensor] = "up to date"

AttributeError: 'DatetimeIndex' object has no attribute 'columns'

In [21]:
logs_joined_unique["Agroscope Berries"]["LeafTemp_Z2_side_int"] = logs_joined_unique["Agroscope Berries"]["LeafTemp_Z2_side_int"].index.drop_duplicates(keep='last')

In [22]:
logs_joined_unique["Agroscope Berries"]["LeafTemp_Z2_side_int"]

DatetimeIndex(['2023-12-12 00:00:00+01:00', '2023-12-12 00:01:00+01:00',
               '2023-12-12 00:02:00+01:00', '2023-12-12 00:03:00+01:00',
               '2023-12-12 00:04:00+01:00', '2023-12-12 00:05:00+01:00',
               '2023-12-12 00:06:00+01:00', '2023-12-12 00:07:00+01:00',
               '2023-12-12 00:08:00+01:00', '2023-12-12 00:09:00+01:00',
               ...
               '2023-12-20 09:17:00+01:00', '2023-12-20 09:18:00+01:00',
               '2023-12-20 09:19:00+01:00', '2023-12-20 09:20:00+01:00',
               '2023-12-20 09:21:00+01:00', '2023-12-20 09:22:00+01:00',
               '2023-12-20 09:23:00+01:00', '2023-12-20 09:24:00+01:00',
               '2023-12-20 09:25:00+01:00', '2023-12-20 09:26:00+01:00'],
              dtype='datetime64[ns, Europe/Zurich]', length=12067, freq=None)

In [15]:
logs_joined_unique["Agroscope Berries"]["LeafTemp_Z2_int_int"].channel_name.unique()

array(['temp_avg', 'temp_min', 'temp_max', 'temp_raw', 'temp_sensor'],
      dtype=object)

# to add: remove duplicate

In [5]:
name = logs_joined_unique["Etchelecu"]["RAIN_EXT"]["sensor_name"].duplicated(keep='first').values
ind = logs_joined_unique["Etchelecu"]["RAIN_EXT"].index.duplicated(keep='first')
mask = [name[i] and ind[i] for i in range(len(name))]
logs_joined_unique["Etchelecu"]["RAIN_EXT"][mask]

,sensor_channel_id,Accumulated rain [mm],integrated [mm · h],gateway_id,local_channel_id,created_at,deleted_at,sensor_name,channel_name,sensor_type,...,serial_number,configuration_json,address,dx,dy,dz,nx,ny,zones_ids,zones_names
2023-12-12 00:01:00+01:00,1145,22.55,0.294403,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-12 00:02:00+01:00,1145,22.55,0.670236,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-12 00:03:00+01:00,1145,22.55,1.046070,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-12 00:04:00+01:00,1145,22.55,1.421903,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-12 00:05:00+01:00,1145,22.55,1.797736,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-20 09:23:00+01:00,1145,64.38,597.513550,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-20 09:24:00+01:00,1145,64.38,598.586548,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-20 09:25:00+01:00,1145,64.38,599.659546,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]
2023-12-20 09:26:00+01:00,1145,64.38,600.732544,31,33,2023-11-21T10:42:55.000000Z,None,RAIN_EXT,rainfall_acc,233,...,unknown,[],81,0,0,0,1,1,[],[]


In [30]:
for i in last_log:
    print(i)
    print(tabulate(last_log[i].items(), headers=["Sensor", "Last log"], tablefmt="github"))
    print("\n")

Etchelecu
| Sensor                | Last log                  |
|-----------------------|---------------------------|
| PAR01_Insolagrin      | 2023-12-18 16:23:00+00:00 |
| PAR02_Insolagrin      | 2023-12-18 16:23:00+00:00 |
| PAR03_Insolagrin      | 2023-12-18 16:23:00+00:00 |
| PAR04_Insolagrin      | 2023-12-18 16:23:00+00:00 |
| PAR01_EXT             | 2023-12-18 16:23:00+00:00 |
| PAR02_EXT             | 2023-12-18 16:23:00+00:00 |
| PAR01_control         | 2023-12-18 16:23:00+00:00 |
| PAR02_control         | 2023-12-18 16:23:00+00:00 |
| GHI_01_EXT            | 2023-12-18 16:23:00+00:00 |
| TRH_top_Insolagrin    | 2023-12-18 16:23:00+00:00 |
| TRH_bottom_Insolagrin | 2023-12-18 16:22:00+00:00 |
| TRH_top_EXT           | 2023-12-18 16:23:00+00:00 |
| TRH_bottom_EXT        | 2023-12-18 16:23:00+00:00 |
| TRH_top_control       | 2023-12-18 16:23:00+00:00 |
| TRH_bottom_control    | 2023-12-18 16:23:00+00:00 |
| RAIN_EXT              | 2023-12-18 16:23:00+00:00 |
| WIND01_EXT      

In [31]:
# time_diff = {}
time_diff_mean = {}
for instal in logs_joined_unique:
    # time_diff[instal] = {}
    time_diff_mean[instal] = {}
    for sensor in logs_joined_unique[instal]:
        time_diff_mean[instal][sensor] = logs_joined_unique[instal][sensor].index.to_series().diff()[1:].mean().total_seconds()
        # time_diff[sensor] = time_diff[sensor][1:]
        # time_diff_mean[sensor] = time_diff[sensor].mean().total_seconds()

    #compute the average time difference between two logs in seconds

    # for sensor in logs_joined_unique:


        #make a bar plot of the average time difference between two logs using plotly
    fig = go.Figure()
    for sensor in logs_joined_unique[instal]:
        fig.add_trace(go.Bar(x=[sensor], y=[time_diff_mean[instal][sensor]]))
    fig.update_layout(title_text='Average time difference between two logs in seconds')
    fig.show()

In [22]:
#make a df, with the name of the sensor, the average time difference between two logs and the last log
df = pd.DataFrame.from_dict(time_diff_mean, orient='index', columns=['time_diff_mean'])
# df['last_log'] = last_log.values()
#add the last log to the df, based on the name of the sensor

last_log


df['last_log'] = df.index.map(last_log)
df

,time_diff_mean,last_log


In [7]:
print(tabulate(df, headers="keys", tablefmt="grid", showindex=True))

+-----------------------+------------------+------------+
|                       |   time_diff_mean | last_log   |
+=======================+==================+============+
| PAR01_Insolagrin      |          62.7901 | up to date |
+-----------------------+------------------+------------+
| PAR02_Insolagrin      |          62.7849 | up to date |
+-----------------------+------------------+------------+
| PAR03_Insolagrin      |          62.7849 | up to date |
+-----------------------+------------------+------------+
| PAR04_Insolagrin      |          62.7796 | up to date |
+-----------------------+------------------+------------+
| PAR01_EXT             |          62.7796 | up to date |
+-----------------------+------------------+------------+
| PAR02_EXT             |          62.7796 | up to date |
+-----------------------+------------------+------------+
| PAR01_control         |          62.7586 | up to date |
+-----------------------+------------------+------------+
| PAR02_contro

In [20]:
last_log

{'PAR01_Insolagrin': 'up to date',
 'PAR02_Insolagrin': 'up to date',
 'PAR03_Insolagrin': 'up to date',
 'PAR04_Insolagrin': 'up to date',
 'PAR01_EXT': 'up to date',
 'PAR02_EXT': 'up to date',
 'PAR01_control': 'up to date',
 'PAR02_control': 'up to date',
 'GHI_01_EXT': 'up to date',
 'TRH_top_Insolagrin': 'up to date',
 'TRH_bottom_Insolagrin': 'up to date',
 'TRH_top_EXT': 'up to date',
 'TRH_bottom_EXT': 'up to date',
 'TRH_top_control': 'up to date',
 'TRH_bottom_control': 'up to date',
 'RAIN_EXT': 'up to date',
 'WIND01_EXT': 'up to date',
 'WIND02_EXT': 'up to date'}